# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
# Load the dotenv extension
%load_ext dotenv 
# Load environment variables from a .env file
%dotenv 


In [2]:
import dask.dataframe as dd

c:\Users\zoomf\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
# Load the environment variable `PRICE_DATA`
price_data_path = os.getenv("PRICE_DATA")

# find the path of all parquet files in the directory `PRICE_DATA`
parquet_files = glob(os.path.join(price_data_path, "**/*.parquet"), recursive = True)


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.
import dask.dataframe as dd
# populate Dask dataframe
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")


In [5]:
# Add lags for Close variable
dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
)

C:\Users\zoomf\AppData\Local\Temp\ipykernel_17260\1941963849.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(


In [6]:
# Add lags for Adj_Close variable
dd_shift = dd_shift.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Adj_Close_lag_1 = x['Adj Close'].shift(1))
)

C:\Users\zoomf\AppData\Local\Temp\ipykernel_17260\69911380.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_shift.groupby('Ticker', group_keys=False).apply(


In [7]:
# Add returns based on Close
dd_returns = dd_shift.assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

In [8]:
# Add hi_lo_range
dd_feat = dd_returns.assign(
    hi_lo_range = lambda x: x['High'] - x['Low'] 
)

In [9]:
dd_feat

,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
npartitions=13078,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
import time

In [11]:
# check if calcluating moving average on Dask is faster than on Pandas
start_ma_on_dask = time.time()

In [12]:
dd_feat_ma = dd_feat.groupby('Ticker').apply(
    lambda x: x.assign(ma_returns = x["returns"].rolling(10).mean())
)

C:\Users\zoomf\AppData\Local\Temp\ipykernel_17260\2379125083.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat_ma = dd_feat.groupby('Ticker').apply(


In [13]:
dd_feat_ma_computed = dd_feat_ma.compute()

In [14]:
end_ma_on_dask = time.time()

In [15]:
dd_feat_ma_computed

Price               Date  Adj Close      Close       High        Low  \
Ticker Ticker                                                          
DOV    DOV    2002-01-02  16.045656  24.842640  24.963299  24.420326   
       DOV    2002-01-03  16.340063  25.298468  25.405722  24.835936   
       DOV    2002-01-04  16.721075  25.888363  25.955397  25.405722   
       DOV    2002-01-07  16.608505  25.714075  26.136387  25.687262   
       DOV    2002-01-08  16.236156  25.137587  25.807922  24.869452   
...                  ...        ...        ...        ...        ...   
CTLT   CTLT   2000-12-22        NaN        NaN        NaN        NaN   
       CTLT   2000-12-26        NaN        NaN        NaN        NaN   
       CTLT   2000-12-27        NaN        NaN        NaN        NaN   
       CTLT   2000-12-28        NaN        NaN        NaN        NaN   
       CTLT   2000-12-29        NaN        NaN        NaN        NaN   

Price               Open     Volume  Year  Close_lag_1  Adj_Close_lag_1  \
Ticker Ticker                                                             
DOV    DOV     24.869452  1022622.0  2002          NaN              NaN   
       DOV     24.876156   989355.0  2002    24.842640        16.045656   
       DOV     25.405722  1319488.0  2002    25.298468        16.340063   
       DOV     25.975506  1060812.0  2002    25.888363        16.721075   
       DOV     25.673855   976377.0  2002    25.714075        16.608505   
...                  ...        ...   ...          ...              ...   
CTLT   CTLT          NaN        NaN  2000          NaN              NaN   
       CTLT          NaN        NaN  2000          NaN              NaN   
       CTLT          NaN        NaN  2000          NaN              NaN   
       CTLT          NaN        NaN  2000          NaN              NaN   
       CTLT          NaN        NaN  2000          NaN              NaN   

Price           returns  hi_lo_range  ma_returns  
Ticker Ticker                                     
DOV    DOV          NaN     0.542973         NaN  
       DOV     0.018349     0.569786         NaN  
       DOV     0.023317     0.549675         NaN  
       DOV    -0.006732     0.449125         NaN  
       DOV    -0.022419     0.938471         NaN  
...                 ...          ...         ...  
CTLT   CTLT         NaN          NaN         NaN  
       CTLT         NaN          NaN         NaN  
       CTLT         NaN          NaN         NaN  
       CTLT         NaN          NaN         NaN  
       CTLT         NaN          NaN         NaN  

[3172924 rows x 13 columns]

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [16]:
# Write your code below.


In [17]:
start_ma_on_pandas = time.time()

In [18]:
# Convert the Dask dataframe to Pandas
dd_feat_computed = dd_feat.compute()

In [19]:
# Add Moving Average on returns on Pandas
dd_feat_computed_ma = dd_feat_computed.groupby('Ticker').apply(
    lambda x: x.assign(ma_returns = x["returns"].rolling(10).mean())
)

In [20]:
end_ma_on_pandas = time.time()

print(f'MA on Dask took {end_ma_on_dask - start_ma_on_dask} seconds.')
print(f'MA on Pandas took {end_ma_on_pandas - start_ma_on_pandas} seconds.')


MA on Dask took 292.9401969909668 seconds.
MA on Pandas took 271.3461389541626 seconds.


In [21]:
dd_feat_computed_ma

Price               Date  Adj Close      Close       High        Low  \
Ticker Ticker                                                          
A      A      2004-01-02  17.353142  20.600859  20.951359  20.586552   
       A      2004-01-05  17.805042  21.137339  21.144491  20.894135   
       A      2004-01-06  18.365395  21.802574  21.816881  21.080114   
       A      2004-01-07  18.600393  22.081545  22.081545  21.673820   
       A      2004-01-08  19.449974  23.090130  23.118740  22.052933   
...                  ...        ...        ...        ...        ...   
ZTS    ZTS    2008-12-24        NaN        NaN        NaN        NaN   
       ZTS    2008-12-26        NaN        NaN        NaN        NaN   
       ZTS    2008-12-29        NaN        NaN        NaN        NaN   
       ZTS    2008-12-30        NaN        NaN        NaN        NaN   
       ZTS    2008-12-31        NaN        NaN        NaN        NaN   

Price               Open     Volume  Year  Close_lag_1  Adj_Close_lag_1  \
Ticker Ticker                                                             
A      A       20.922747  2839478.0  2004          NaN              NaN   
       A       21.101574  5234531.0  2004    20.600859        17.353142   
       A       21.137339  4921100.0  2004    21.137339        17.805042   
       A       21.816881  4231606.0  2004    21.802574        18.365395   
       A       22.095852  7320487.0  2004    22.081545        18.600393   
...                  ...        ...   ...          ...              ...   
ZTS    ZTS           NaN        NaN  2008          NaN              NaN   
       ZTS           NaN        NaN  2008          NaN              NaN   
       ZTS           NaN        NaN  2008          NaN              NaN   
       ZTS           NaN        NaN  2008          NaN              NaN   
       ZTS           NaN        NaN  2008          NaN              NaN   

Price           returns  hi_lo_range  ma_returns  
Ticker Ticker                                     
A      A            NaN     0.364807         NaN  
       A       0.026042     0.250357         NaN  
       A       0.031472     0.736767         NaN  
       A       0.012795     0.407724         NaN  
       A       0.045675     1.065807         NaN  
...                 ...          ...         ...  
ZTS    ZTS          NaN          NaN         NaN  
       ZTS          NaN          NaN         NaN  
       ZTS          NaN          NaN         NaN  
       ZTS          NaN          NaN         NaN  
       ZTS          NaN          NaN         NaN  

[3172924 rows x 13 columns]

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

> No, it was not necessary to convert the Dask dataframe to Pandas. It's possible to use Dask lazily execute operations and use .compute() only when we need to convert to Pandas.

> If dataset is large using Dask could be more efficient and faster as Dask will optimize calculations and utilize multiple CPUs.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.